In [0]:
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import time
from PIL import Image

from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [0]:
df = pd.read_json("/content/Indian_Number_plates.json", lines=True)

df.head()

In [0]:
df['annotation'][0]

In [0]:
os.mkdir("Indian Number Plates")

In [0]:
dataset = dict()
dataset["image_name"] = list()
dataset["image_width"] = list()
dataset["image_height"] = list()
dataset["top_x"] = list()
dataset["top_y"] = list()
dataset["bottom_x"] = list()
dataset["bottom_y"] = list()

counter = 0
for index, row in df.iterrows():
    img = urllib.request.urlopen(row["content"])
    img = Image.open(img)
    img = img.convert('RGB')
    img.save("Indian Number Plates/licensed_car{}.jpeg".format(counter), "JPEG")
    
    dataset["image_name"].append("licensed_car{}".format(counter))
    
    data = row["annotation"]
    
    dataset["image_width"].append(data[0]["imageWidth"])
    dataset["image_height"].append(data[0]["imageHeight"])
    dataset["top_x"].append(data[0]["points"][0]["x"])
    dataset["top_y"].append(data[0]["points"][0]["y"])
    dataset["bottom_x"].append(data[0]["points"][1]["x"])
    dataset["bottom_y"].append(data[0]["points"][1]["y"])
    
    counter += 1
print("Downloaded {} car images.".format(counter))

In [0]:
df=pd.DataFrame(dataset)
df.head()

In [0]:
df["image_name"]=df["image_name"] + ".jpeg"


In [0]:
df.head()

In [0]:
df.drop(["image_width", "image_height"], axis=1, inplace=True)
df.head()

In [0]:
import numpy as np
test_samples=np.random.randint(0,len(df),5)

In [0]:
test_samples

In [0]:
reduced_df=df.drop(test_samples,axis=0)

In [0]:
w=224
h=224
channels=3

import cv2

def show_img(index):
  image=cv2.imread('Indian Number Plates/'+df["image_name"].iloc[index])
  image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image=cv2.resize(image,dsize=(w,h))

  tx=int(df['top_x'].iloc[index]*w)
  ty=int(df['top_y'].iloc[index]*h)
  bx=int(df['bottom_x'].iloc[index]*w)
  by=int(df['bottom_y'].iloc[index]*h)

  image=cv2.rectangle(image,(tx,ty),(bx,by),(0,0,255),5)
  plt.imshow(image)
  plt.show()


In [0]:
show_img(5)

In [0]:
datagen=ImageDataGenerator(rescale=1./255,validation_split=0.1)


In [0]:
train_generator=datagen.flow_from_dataframe(reduced_df,directory='/content/Indian Number Plates',
                                            x_col='image_name',
                                            y_col=['top_x','top_y','bottom_x','bottom_y'],
                                            target_size=(h,w),
                                            class_mode='raw',
                                            subset='training')

In [0]:
validation_generator=datagen.flow_from_dataframe(reduced_df,directory='/content/Indian Number Plates',
                                                 x_col='image_name',
                                                 y_col=['top_x','top_y','bottom_x','bottom_y'],
                                                 target_size=(h,w),
                                                 class_mode='raw',
                                                 batch_size=32,
                                                 subset='validation')

In [0]:
model = Sequential()
model.add(VGG16(weights="imagenet", include_top=False, input_shape=(h, w, channels)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

model.layers[-6].trainable = False

model.summary()

In [0]:
STEP_SIZE_TRAIN = int(np.ceil(train_generator.n / train_generator.batch_size))
STEP_SIZE_VAL = int(np.ceil(validation_generator.n / validation_generator.batch_size))

print("Train step size:", STEP_SIZE_TRAIN)
print("Validation step size:", STEP_SIZE_VAL)

train_generator.reset()
validation_generator.reset()

In [0]:
adam = Adam(lr=0.0005)
model.compile(optimizer=adam, loss="mse")

In [0]:
history = model.fit_generator(train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VAL,
    epochs=30)

In [0]:
model.evaluate_generator(validation_generator, steps=STEP_SIZE_VAL)


In [0]:
for idx, row in df.iloc[test_samples].iterrows():
   img = cv2.resize(cv2.imread("Indian Number Plates/" + row[0]) / 255.0, dsize=(w,h))
   y_hat = model.predict(img.reshape(1,w,h, 3)).reshape(-1) * w
    
   xt, yt = y_hat[0], y_hat[1]
   xb, yb = y_hat[2], y_hat[3]
    
   img = cv2.cvtColor(img.astype(np.float32), cv2.COLOR_BGR2RGB)
   image = cv2.rectangle(img, (xt, yt), (xb, yb), (0, 0, 255), -1)
   plt.imshow(image)
   plt.show()
